In [1]:
import numpy as np
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install pandas pyarrow fastparquet

In [2]:
flights_sample_small=pd.read_csv('./tables/flights_sample_small.csv')

In [3]:
flights_sample_medium=pd.read_csv('./tables/flights_sample_medium.csv')

In [4]:
flights_sample_medium.to_parquet('./tables/flights_sample_medium.parquet')

In [5]:
flights_sample_medium=pd.read_parquet('./tables/flights_sample_medium.parquet')

In [6]:
flights_sample_large=pd.read_csv('./tables/flights_sample_large.csv')

In [7]:
flights_sample_large.to_parquet('./tables/flights_sample_large.parquet')

In [8]:
flights_sample_large = pd.read_parquet('./tables/flights_sample_large.parquet')

## 1.   knock-on Feature

In [9]:
X=flights_sample_small

In [10]:
X1=flights_sample_medium

In [11]:
X2=flights_sample_large

In [12]:
def making_table_ready(X):   
    X['crs_dep_time'] = X['crs_dep_time'].astype(str).str.zfill(4)
    X['crs_arr_time'] = X['crs_arr_time'].astype(str).str.zfill(4)

    def get_hour_min(x):    
        return str(int(x[:2]))+':'+str(int(x[-2:]))

    X['crs_dep_time'] = X['crs_dep_time'].apply(lambda x: get_hour_min(x))
    X['crs_arr_time'] = X['crs_arr_time'].apply(lambda x: get_hour_min(x))

    #adding zero so that each time value consists of 4 digits
    X['crs_dep_time'] = X['crs_dep_time'].str.replace('24:0', '00:00')
    X['crs_arr_time'] = X['crs_arr_time'].str.replace('24:0', '00:00')


    # convert time columns to datetime
    X['crs_dep_time'] = pd.to_datetime(X['crs_dep_time'], format='%H:%M')
    X['crs_arr_time'] = pd.to_datetime(X['crs_arr_time'], format='%H:%M')
    
    X['knock-on']=np.nan
    
    return X

In [13]:
def table_with_nock_on(table):
    #unique tail nums and dates
    tail_num_val=list(table['tail_num'].unique())
    fl_date_val=list(table['fl_date'].unique())
    
    #define a function to filter the table
    def filtered_sorted(tail_num_val,fl_date_val):
        a=X[(X['tail_num'] == tail_num_val) & (X['fl_date'] == fl_date_val)].sort_values('crs_dep_time').reset_index()
        return a
    
    for i in tail_num_val:
        #print (i)
        for j in fl_date_val:
            #print (j)
            df=filtered_sorted(i,j)
            #print (df)
            for k in range(len(df)-1):
                a=df.loc[k+1, 'index'] #main_index_table                
                c=(df.loc[k+1, 'crs_dep_time'])
                d=(df.loc[k, 'crs_arr_time'])
                table.loc[a,'knock-on']=c-d
                table.loc[a,'knock-on']=table.loc[a,'knock-on']
                
    #table['crs_dep_time']=table['crs_dep_time'].dt.time
    #table['crs_arr_time']=table['crs_arr_time'].dt.time
    
    #table['knock-on'] = table.apply(lambda r: np.nan if type(r['knock-on']) == float else r['knock-on'], axis=1)

    #table['knock-on'] = (pd.Timestamp('now').normalize() + table['knock-on']).dt.time
    return table

In [14]:
making_table_ready(X)
table_with_nock_on(X)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,knock-on
0,2019-01-01,AA,5677,OH,N576NN,11423,DSM,"Des Moines, IA",11057,CLT,"Charlotte, NC",1900-01-01 06:20:00,1900-01-01 09:56:00,156,815,-7.0,NaN
1,2019-01-01,AA,5606,OH,N575NN,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",1900-01-01 21:57:00,1900-01-01 23:11:00,74,157,-3.0,0 days 00:30:00
2,2019-01-01,AA,5607,OH,N254PS,10529,BDL,"Hartford, CT",11278,DCA,"Washington, DC",1900-01-01 11:51:00,1900-01-01 13:25:00,94,313,6.0,0 days 00:25:00
3,2019-01-01,AA,5607,OH,N254PS,11278,DCA,"Washington, DC",10529,BDL,"Hartford, CT",1900-01-01 10:09:00,1900-01-01 11:26:00,77,313,14.0,0 days 01:01:00
4,2019-01-01,AA,5608,OH,N710PS,11278,DCA,"Washington, DC",11996,GSP,"Greer, SC",1900-01-01 20:00:00,1900-01-01 21:34:00,94,396,-12.0,0 days 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19517,2019-01-01,DL,830,DL,N909DA,10397,ATL,"Atlanta, GA",11066,CMH,"Columbus, OH",1900-01-01 19:03:00,1900-01-01 20:33:00,90,447,-9.0,0 days 00:50:00
19518,2019-01-01,DL,831,DL,N344NW,11278,DCA,"Washington, DC",13487,MSP,"Minneapolis, MN",1900-01-01 16:56:00,1900-01-01 18:57:00,181,931,-24.0,0 days 00:46:00
19519,2019-01-01,DL,832,DL,N659DL,14869,SLC,"Salt Lake City, UT",11278,DCA,"Washington, DC",1900-01-01 09:32:00,1900-01-01 15:32:00,240,1851,-18.0,NaN
19520,2019-01-01,DL,833,DL,N365DN,11433,DTW,"Detroit, MI",14679,SAN,"San Diego, CA",1900-01-01 08:30:00,1900-01-01 10:47:00,317,1956,-25.0,NaN


In [15]:
making_table_ready(X1)
table_with_nock_on(X1)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,knock-on
0,2019-01-01,AA,5677,OH,N576NN,11423,DSM,"Des Moines, IA",11057,CLT,"Charlotte, NC",1900-01-01 06:20:00,1900-01-01 09:56:00,156,815,-7.0,NaN
1,2019-01-01,AA,5628,OH,N260JS,14492,RDU,"Raleigh/Durham, NC",11278,DCA,"Washington, DC",1900-01-01 10:36:00,1900-01-01 11:45:00,69,227,-18.0,0 days 00:30:00
2,2019-01-01,AA,5629,OH,N589NN,11057,CLT,"Charlotte, NC",12217,HSV,"Huntsville, AL",1900-01-01 22:40:00,1900-01-01 23:10:00,90,333,62.0,0 days 00:25:00
3,2019-01-01,AA,5632,OH,N598NN,11057,CLT,"Charlotte, NC",13296,MHT,"Manchester, NH",1900-01-01 09:49:00,1900-01-01 12:03:00,134,737,-12.0,0 days 01:01:00
4,2019-01-01,AA,5632,OH,N598NN,13296,MHT,"Manchester, NH",11057,CLT,"Charlotte, NC",1900-01-01 12:39:00,1900-01-01 15:13:00,154,737,0.0,0 days 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145359,2019-01-07,UA,3656,YX,N744YX,11618,EWR,"Newark, NJ",11057,CLT,"Charlotte, NC",1900-01-01 16:45:00,1900-01-01 18:48:00,123,529,2.0,NaN
145360,2019-01-07,UA,3657,YX,N743YX,11057,CLT,"Charlotte, NC",12266,IAH,"Houston, TX",1900-01-01 18:30:00,1900-01-01 20:35:00,185,912,0.0,NaN
145361,2019-01-07,UA,3658,YX,N750YX,13487,MSP,"Minneapolis, MN",11618,EWR,"Newark, NJ",1900-01-01 12:25:00,1900-01-01 16:15:00,170,1008,-7.0,NaN
145362,2019-01-07,UA,3659,YX,N863RW,10781,BTR,"Baton Rouge, LA",12266,IAH,"Houston, TX",1900-01-01 05:25:00,1900-01-01 06:43:00,78,253,-27.0,NaN


In [16]:
making_table_ready(X2)
table_with_nock_on(X2)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,knock-on
0,2018-12-24,HA,154,HA,N486HA,12982,LIH,"Lihue, HI",12173,HNL,"Honolulu, HI",1900-01-01 16:10:00,1900-01-01 16:46:00,36,102,-7.0,NaN
1,2018-12-27,WN,6776,WN,N8624J,13796,OAK,"Oakland, CA",14679,SAN,"San Diego, CA",1900-01-01 09:00:00,1900-01-01 10:30:00,90,446,NaN,0 days 00:30:00
2,2018-12-24,NK,423,NK,N636NK,10721,BOS,"Boston, MA",13204,MCO,"Orlando, FL",1900-01-01 19:59:00,1900-01-01 23:26:00,207,1121,-1.0,0 days 00:25:00
3,2018-12-24,HA,152,HA,N477HA,13830,OGG,"Kahului, HI",12402,ITO,"Hilo, HI",1900-01-01 16:15:00,1900-01-01 16:54:00,39,121,-7.0,0 days 01:01:00
4,2018-12-24,HA,153,HA,N493HA,12173,HNL,"Honolulu, HI",12982,LIH,"Lihue, HI",1900-01-01 14:01:00,1900-01-01 14:46:00,45,102,-13.0,0 days 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884433,2018-01-14,UA,2021,UA,N77542,12264,IAD,"Washington, DC",14843,SJU,"San Juan, PR",1900-01-01 08:15:00,1900-01-01 13:00:00,225,1571,26.0,NaN
884434,2018-01-14,UA,2017,UA,N14704,12266,IAH,"Houston, TX",14683,SAT,"San Antonio, TX",1900-01-01 12:03:00,1900-01-01 13:11:00,68,191,12.0,NaN
884435,2018-01-14,UA,2015,UA,N69885,11618,EWR,"Newark, NJ",13930,ORD,"Chicago, IL",1900-01-01 18:38:00,1900-01-01 20:15:00,157,719,0.0,NaN
884436,2018-01-14,UA,2014,UA,N23721,11503,EGE,"Eagle, CO",13930,ORD,"Chicago, IL",1900-01-01 11:34:00,1900-01-01 15:14:00,160,1007,-19.0,NaN


In [17]:
X.to_parquet('./tables/flights_sample_small_knock_on.parquet', index=False)
X1.to_parquet('./tables/flights_sample_medium_knock_on.parquet', index=False)
X2.to_parquet('./tables/flights_sample_large_knock_on.parquet', index=False)

##  2. Origin and Departure plane traffic Feature

In [38]:
Y = pd.read_parquet('./tables/flights_sample_small_knock_on.parquet')
Y1 = pd.read_parquet('./tables/flights_sample_medium_knock_on.parquet')
Y2 = pd.read_parquet('./tables/flights_sample_large_knock_on.parquet')

In [23]:
def making_table_ready_2(X):   
    X['crs_dep_time'] = X['crs_dep_time'].astype(str).str.zfill(4)
    X['crs_arr_time'] = X['crs_arr_time'].astype(str).str.zfill(4)

    def get_hour_min(x):    
        return str(int(x[:2]))+':'+str(int(x[-2:]))

    X['crs_dep_time'] = X['crs_dep_time'].apply(lambda x: get_hour_min(x))
    X['crs_arr_time'] = X['crs_arr_time'].apply(lambda x: get_hour_min(x))

    #adding zero so that each time value consists of 4 digits
    X['crs_dep_time'] = X['crs_dep_time'].str.replace('24:0', '00:00')
    X['crs_arr_time'] = X['crs_arr_time'].str.replace('24:0', '00:00')


    # convert time columns to datetime
    X['crs_dep_time'] = pd.to_datetime(X['crs_dep_time'], format='%H:%M')
    X['crs_arr_time'] = pd.to_datetime(X['crs_arr_time'], format='%H:%M')

   
    X['origin_plane_traffic']=np.nan
    X['dest_plane_traffic']=np.nan
    
    return X
    #Convert dates to string type
    X['crs_dep_time']=X['crs_dep_time'].apply(lambda x: x.strftime('%H:%M:%S'))
    X['crs_arr_time']=X['crs_arr_time'].apply(lambda x: x.strftime('%H:%M:%S'))

    return X

In [24]:
making_table_ready_2(Y)
making_table_ready_2(Y1)
making_table_ready_2(Y2)
Y

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,knock-on,origin_plane_traffic,dest_plane_traffic
0,2019-01-01,AA,5677,OH,N576NN,11423,DSM,"Des Moines, IA",11057,CLT,"Charlotte, NC",1900-01-01 19:00:00,1900-01-01 19:00:00,156,815,-7.0,NaT,NaN,NaN
1,2019-01-01,AA,5606,OH,N575NN,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",1900-01-01 19:00:00,1900-01-01 19:00:00,74,157,-3.0,0 days 00:30:00,NaN,NaN
2,2019-01-01,AA,5607,OH,N254PS,10529,BDL,"Hartford, CT",11278,DCA,"Washington, DC",1900-01-01 19:00:00,1900-01-01 19:00:00,94,313,6.0,0 days 00:25:00,NaN,NaN
3,2019-01-01,AA,5607,OH,N254PS,11278,DCA,"Washington, DC",10529,BDL,"Hartford, CT",1900-01-01 19:00:00,1900-01-01 19:00:00,77,313,14.0,0 days 01:01:00,NaN,NaN
4,2019-01-01,AA,5608,OH,N710PS,11278,DCA,"Washington, DC",11996,GSP,"Greer, SC",1900-01-01 19:00:00,1900-01-01 19:00:00,94,396,-12.0,0 days 00:40:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19517,2019-01-01,DL,830,DL,N909DA,10397,ATL,"Atlanta, GA",11066,CMH,"Columbus, OH",1900-01-01 19:00:00,1900-01-01 19:00:00,90,447,-9.0,0 days 00:50:00,NaN,NaN
19518,2019-01-01,DL,831,DL,N344NW,11278,DCA,"Washington, DC",13487,MSP,"Minneapolis, MN",1900-01-01 19:00:00,1900-01-01 19:00:00,181,931,-24.0,0 days 00:46:00,NaN,NaN
19519,2019-01-01,DL,832,DL,N659DL,14869,SLC,"Salt Lake City, UT",11278,DCA,"Washington, DC",1900-01-01 19:00:00,1900-01-01 19:00:00,240,1851,-18.0,NaT,NaN,NaN
19520,2019-01-01,DL,833,DL,N365DN,11433,DTW,"Detroit, MI",14679,SAN,"San Diego, CA",1900-01-01 19:00:00,1900-01-01 19:00:00,317,1956,-25.0,NaT,NaN,NaN


In [36]:
def origin_dest_plane_traffic(fl_date, tail_num, crs_dep_time, crs_arr_time, df):
    # Get the row of the specific flight
    flight_row = df.loc[(df['fl_date'] == fl_date) & (df['tail_num'] == tail_num) & (df['crs_dep_time'] == crs_dep_time)]
    
    # Get origin and destination airports from the flight row
    origin_airport_id = flight_row['origin_airport_id'].iloc[0]
    dest_airport_id = flight_row['dest_airport_id'].iloc[0]
    
    # Set the time difference
    time_diff = pd.Timedelta(minutes=30)
    
    # Filter the flights based on origin airport and time
    origin_flights = df[(df['fl_date'] == fl_date) & (df['origin_airport_id'] == origin_airport_id) & 
                        (pd.to_datetime(df['crs_dep_time']) >= pd.to_datetime(crs_dep_time) - time_diff) & 
                        (pd.to_datetime(df['crs_dep_time']) <= pd.to_datetime(crs_dep_time) + time_diff)]
    origin_traffic = len(origin_flights)
    
    # Filter the flights based on destination airport and time
    dest_flights = df[(df['fl_date'] == fl_date) & (df['dest_airport_id'] == dest_airport_id) & 
                      (pd.to_datetime(df['crs_arr_time']) >= pd.to_datetime(crs_arr_time) - time_diff) & 
                      (pd.to_datetime(df['crs_arr_time']) <= pd.to_datetime(crs_arr_time) + time_diff)]
    dest_traffic = len(dest_flights)
    
    # Return a list for origin and arrival plane traffic
    return [origin_traffic, dest_traffic]

In [37]:
def get_plane_traffic(row, df):
    a = row['fl_date']
    b = row['tail_num']
    c = row['crs_dep_time'] 
    d = row['crs_arr_time']
    e = origin_dest_plane_traffic(a, b, c, d, df)
    return pd.Series({'origin_plane_traffic': e[0], 'dest_plane_traffic': e[1]})

def plane_traffic(df):
    df[['origin_plane_traffic', 'dest_plane_traffic']] = df.apply(lambda row: get_plane_traffic(row, df), axis=1)
    return df

In [31]:
plane_traffic(Y)

KeyboardInterrupt: 

In [35]:
plane_traffic(Y1)

KeyboardInterrupt: 

In [34]:
Y.to_parquet('./tables/flights_sample_small_with_features', index=False)

In [39]:
Y1.to_parquet('./tables/flights_sample_medium_with_features.parquet', index=False)